In [1]:
import numpy as np
import pandas as pd
import requests
from PIL import Image
from sklearn.cluster import KMeans, MiniBatchKMeans

In [2]:
path = r'./Chungking-Express-14.jpg'

In [3]:
def online_image_loader(url):
    response = requests.get(url, stream=True).raw
    try:
        image = Image.open(response)
    except:
        raise Exception('Image not found, try another URL')  
    return image

In [4]:
def local_image_loader(path):
    try:
        image = Image.open(path)
    except:
        raise Exception('Image not found, try another path')
    return image

In [5]:
def flatten_image(image):
    df = pd.DataFrame(list(image.getdata()),columns=list(image.getbands()))
    return df

In [6]:
img_local = local_image_loader(path)
# url = input('Enter image url: ')
# img_online = online_image_loader(url)

In [7]:
flat_local = flatten_image(img_local)
# flat_online = flatten_image(img_online)

In [1]:
class Clusterer():
    def __init__(self, df):
        self.df = df
        self.models = {}

    def KM(self):
        self.models['KMeans'] = KMeans(n_clusters=8,n_init='auto').fit(self.df)
        return self.models['KMeans']
    
    def MBKM(self):
        self.models['MiniBatchKMeans'] = MiniBatchKMeans(n_clusters=8,n_init='auto').fit(self.df)
        return self.models['MiniBatchKMeans']

In [9]:
def generate_rgb_palette(model):
    palette = model.cluster_centers_ / 255
    return palette

def rgb_to_hex(rgb_value):
    r, g, b = rgb_value[0], rgb_value[1], rgb_value[2]
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

def generate_hex_palette(model):
    palette = model.cluster_centers_.round().astype(int)
    palette = [rgb_to_hex(p) for p in palette]
    return palette

def hex_to_rgb(hex_value):
  h = hex_value.lstrip('#')
  return tuple(int(h[i:i + 2], 16) / 255.0 for i in (0, 2, 4))

In [13]:
model = Clusterer(flat_local).KM()

In [ ]:
rgb_palette = generate_rgb_palette(model)
hex_palette = generate_hex_palette(model)

In [ ]:
rgb_palette

In [ ]:
for color in hex_palette:
    print(color)